# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

План работы: 

-  Изучение общей информации 
-  Предобработка данных
-  Ответы на вопросы задания
-  Общий вывод

Входные данные от банка — статистика о платёжеспособности клиентов.

Описание данных:

- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [75]:
import pandas as pd

In [76]:
credit = pd.read_csv('/datasets/data.csv')
credit.info()
credit.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

Видим, что есть столбцы с пропущенными значениями и типами float64, а также видим, что все названия столбцов написаны корректно. Также сразу можно обнаржуить, что не все столбцы таблицы нам нужны для анализа. Нужные нам столбцы: children, family_status, debt, total_income, purpose.

In [77]:
print(credit.head(10))

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   
5         0    -926.185831         27    высшее             0   
6         0   -2879.202052         43    высшее             0   
7         0    -152.779569         50   СРЕДНЕЕ             1   
8         2   -6929.865299         35    ВЫСШЕЕ             0   
9         0   -2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0    

Видим, что в столбце days_employed есть отрицательные и невозможно большие значения (нам этот столбец не нужен, поэтому его не трогаем)
Если бы был нужен я бы запросил информацию о столбце из источника, затем удалил минусы (скорей всего ошибка), а с невозможными значениями не знаю что делать (удлил бы наверное)
education: разный регистр

### Вывод: 
Видим большую таблицу из которой нам чтобы ответить на поставленные вопросы нужны только столбцы: 'children', 'family_status', 'debt', 'total_income', 'purpose'. В названиях столбцов проблем нет. Видим, что в столбцах days_employed (ненужный для ответа на вопросы) и total_income (нужный для ответа на вопросы) есть пропущенные значения.
Если бы работалит со всей таблицей, выделили бы еще отрицательные  и невозможно большие значения в столбце days_employed.
В столбце education видим, что текст имеет разный регистр - необходимо сделать данные этого столбца нижнего регистра через str.lower() или использовать только столбец education_id вместо базового education.
Возможные причины пропусков в столбце days_employed на мой взгляд случайные так как значение в нем независит от значений в други столбцах, но пропуски в столбце total_income наверняка возникли из-за пустых значений days_employed (думаю не наоборот, так как стаж может быть, но заработок 0, наоборот же быть не может, если мы рассуждаем о статистике банка).

### Шаг 2. Предобработка данных

### Обработка пропусков

In [78]:
display((credit[credit['total_income'].isnull()].head())) # Смотри строчик с пропущенными значениями

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


In [79]:
credit[credit['total_income'].isna()].count() # Подсчет пропущенных значений по столбцам

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64

In [80]:
credit.duplicated().sum()       # Делал сначала все этапы по порядку этапы,
                                # но после замены пропусков решил, что корректнее удалить
                                # дубликаты в изначальной таблице, так как в дальнейшем
                                # мы заменяем значения в таблицах по которым с большей вероятностью
                                # можно найти дубликаты (знаю, что можно было не менять столбец days_employed)
                                # но по сложности тоже самое выходит

54

In [81]:
credit = credit.drop_duplicates().reset_index(drop = True) # Удаление дубликатов
credit.duplicated().sum() # Проверка

0

Видим, что в столбце total_income есть пропущенные значения. Пропущенно примерно 10% значений, это слишком много чтобы можно было просто выкинуть. Необходимо заполнить пустые значения в столбцах total_income, для этого возьмем стоблец 'income_type', определим его уникальные значения, посчитаем медианы для них и заменим соответствующими мединами пустые значения в столбце total_income.

In [82]:
credit['income_type'].value_counts() # Смотрим все типы занятости для того, чтобы взять по ним медианы вручную.

сотрудник          11091
компаньон           5080
пенсионер           3837
госслужащий         1457
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [83]:
income_median = credit.groupby('income_type')['total_income'].median() # Выделяем медианные значения дохода в зависимости от типа занятости
credit_new = credit.merge(income_median, on ='income_type', how = 'left')  # Сменил подстановку в ручную на группировку
display(credit_new)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income_x,purpose,total_income_y
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,142594.396847
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,142594.396847
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,142594.396847
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,142594.396847
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,118514.486412
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21466,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем,172357.950966
21467,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем,118514.486412
21468,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость,142594.396847
21469,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля,142594.396847


In [84]:
credit_new.set_axis(['children', 'days_employed', 'dob_years', 'education', 'education_id','family_status', 'family_status_id', 'gender', 'income_type', 'debt','total_income', 'purpose', 'total_income_fillna'], axis = 'columns', inplace = True) # Установоили название столбцов 
credit_new.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose', 'total_income_fillna'],
      dtype='object')

In [85]:
credit_new['total_income'] = credit_new['total_income'].fillna(credit_new['total_income_fillna']) # Заполнили пропуски в 'total_income_start' значениями из 'total_income_fillna'

In [86]:
credit_new.isna().sum() # Проверка

children                  0
days_employed          2120
dob_years                 0
education                 0
education_id              0
family_status             0
family_status_id          0
gender                    0
income_type               0
debt                      0
total_income              0
purpose                   0
total_income_fillna       0
dtype: int64

In [ ]:
credit_new.info()

In [ ]:
credit_new['children'].value_counts() # Проверим уникальные значения в столбце children

Видим отрицательное и слишком большое значения, это скорей всего ошибки, их нужно исправить, вероятнее всего ошибки в выгрузке или кто-то случайно добавил - или 0.
Уберём - у 1 и 0 у 20.

In [ ]:
credit_new['children'] = credit_new['children'].replace(-1, 1) # Замена ошибочных значений.
credit_new['children'] = credit_new['children'].replace(20, 2)
display(credit_new['children'].value_counts())

In [ ]:
credit_new.loc[:, ['dob_years', 'days_employed']] # Смотрим на данные в столбце days_employed и видим, что перед заполнением пропусков необходимо их обработать (убрать минусы и невозможные значения, в которых даже если разделить значение на 10 оно все равно большое)

In [ ]:
credit_new['days_employed'].max()

In [ ]:
credit_new['days_employed'] = credit_new['days_employed'].abs()
display(credit_new['days_employed'])
credit_new.info()

### Вывод
Изначально в таблице было больше столбцов, чем нужно - отобрали нужные столбцы и сделали из них таблицу. Были обнаружены пропуски в изначальных данных:  пропуски (NaN) ,возможные причины - ошибки при выгрузке, потеря данных или их отсутствие. Пропуски были заполнены исходя из того, что медианный доход для разных типов занятости позволит сделать анализ более точным. Медиана а не среднее, так как большой разброс значений. 

### Замена типа данных

In [ ]:
credit_new['total_income'] = credit_new['total_income'].astype('int') # Замена вещественного типа данных
credit_new.info()                                                      # на целочисленный.

Обычная замена типа данных, ничего особенного вроде.

### Обработка дубликатов

In [ ]:
print(credit.duplicated().sum()) # Поиск дубликатов в полной таблице
print(credit_new.duplicated().sum()) # Поиск дубликатов в целевой таблице

Дубликаты необходимо удалять в первоночальной таблице, с изначальными данными, так как у нас в исправленных таблицах пропуски (по которым как раз можно в большей степени судить являются строчки дубликатами или нет) заполнены медианными значениями. Следовательно необходимо вернутся назад и сначала удалить дубликаты.

СЕЙЧАС ПРОДУБЛИРУЮ ТО, ЧТО ПЕРЕНЕС В НАЧАЛО (ПОЧЕМУ ТАК СДЕЛАЛ НАПИСАНО СТРОКОЙ ВЫШЕ)

In [ ]:
print(credit.duplicated().sum())# Делал сначала все этапы по порядку этапы,
                                # но после замены пропусков решил, что корректнее удалить
                                # дубликаты в изначальной таблице, так как в дальнейшем
                                # мы заменяем значения в таблицах по которым с большей вероятностью
                                # можно найти дубликаты (знаю, что можно было не менять столбец days_employed)
                                # но по сложности тоже самое выходит

In [ ]:
credit = credit.drop_duplicates().reset_index(drop = True) # Удаление дубликатов
print(credit.duplicated().sum()) # Проверка

### Вывод

Для поиска дубликатов использовался метод .duplicated, для удаления drop_duplicates.reset_index(drop = True). Причина: потому, что удобно, работает и в голову не приходило альтернатив.
Возможные причины появления дубликатов: ошибки в выгрузке, реальное точное совпадение данных (очень маловероятно), ошибки в формировании файла.

### Лемматизация

In [ ]:
credit_new['purpose'].value_counts() # Смотрим уникальные значения в столбце 'purpose' для выделения лем.

In [ ]:
from pymystem3 import Mystem # Получаем лемматизатор на русском
m = Mystem()
def lemma(purpose):          # Пишем функцию для вывода лем по столбцам
    lem = m.lemmatize(purpose)
    return lem

credit_new['lemmas_purpose'] = credit_new['purpose'].apply(lemma) # Применяем метода apply для вызова нашей функции
display(credit_new['lemmas_purpose'].head())                         # Смотрим, что получилось

from collections import Counter                                     # Смотрим количество уникальных значений
all_lemms = credit_new['lemmas_purpose'].sum()
Counter(all_lemms)

### Вывод
Отлемматизировали данные в столбце purpose, видим, что много похожих целей кредита, делаем вывод, что по ним нужно сгруппировать цели.

### Категоризация данных

In [ ]:
def lemma_group(lemmas_purpose):                   # Пишем функцию для выделения категорий для столбца с лемами цели кредита
        if "свадьба" in lemmas_purpose:            # В ручную выбрал категории из списка в предидущем этапе goal_credit['purpose'].value_counts()
            return "свадьба"
        elif 'недвижимость' in lemmas_purpose:
            return 'недвижимость'
        elif 'жилье' in lemmas_purpose:
            return 'жилье'
        elif 'автомобиль' in lemmas_purpose:
            return 'автомобиль'
        elif 'образование' in lemmas_purpose:
            return 'образование'
        
credit_new['lemmas_group'] = credit_new['lemmas_purpose'].apply(lemma_group) # Применили метод 
display(credit_new['lemmas_group'].value_counts())                              # Посмотрели уникальные значения и их количество

Проанализирова уникальные значения столбца "purpose" я выбрал следующие леммы 
1) свадьба (где целью является свадьба, есть данная лемма)
2) недвижимость и жилье (данные леммы я разделил, так как на мой вгляд под жильем понимается объект в котором клиенты живут, а под недвижимостью объекты которые используют для иных целей)
3) автомобиль (где целью является операция с автомобилем, есть данная лемма)
4) образование (где целью является образование, есть данная лемма)

In [ ]:
print(credit_new['total_income'].min())     # Считаю медиану, минимальное и максимальное значения
print(credit_new['total_income'].max())     # для дальнейшей категоризации ежемесячных доходов
print(credit_new['total_income'].median())

In [ ]:
def income(total_income):         # Выделил классы по ежемесячному доходу шаг брал 100 000 исходя из медианы
    if total_income <= 100000:
        return 'бедные'
    if total_income <= 200000:
        return 'средний класс '
    if total_income <= 300000:
        return 'богатые'
    return 'зажиточные'

In [ ]:
credit_new['income_group'] = credit_new['total_income'].apply(income) # Применил метод и функцию 
display(credit_new)                           
goal_income = credit_new['income_group'].value_counts()                # Проверка

In [ ]:
goal_income_debt = credit_new.groupby('income_group')['debt'].sum().sort_values(ascending=False)  # Сгрупировал таблицу по группе дохода
display(goal_income_debt)                                                                            # и посчитал количество просроченных кредитов по этим группам

In [ ]:
credit_new.income_group.value_counts()

Второй сопсоб группировки:

In [ ]:
credit_new_income_qcut = pd.qcut(credit_new.total_income, 5).value_counts().sort_values(ascending = True)
credit_new_income_qcut

In [ ]:
def qcut_income(total_income):
    if 20666.999 < total_income <= 98554.0:
        return "бедные"
    elif 98554.0 < total_income <= 132123.0:
        return 'нижний средний класс'
    elif 132123.0 < total_income <= 161287.0:
        return 'средний класс'
    elif 161287.0 < total_income <= 214565.0:
        return 'верхний средний класс'
    elif 214565.0 < total_income <= 2265604.0:
        return 'богатые'

In [ ]:
credit_new['qcut_income'] = credit_new['total_income'].apply(qcut_income)
display(credit_new)
credit_new['qcut_income'].value_counts()

In [ ]:
qcut_income_pivot = credit_new.pivot_table(index = ['qcut_income'], columns = 'debt', values = 'education_id', aggfunc = 'count')
qcut_income_pivot

In [ ]:
qcut_income_pivot['coef'] = qcut_income_pivot[1] / qcut_income_pivot.sum(axis =1) # Втопрой способ категоризации
qcut_income_pivot.sort_values(by = 'coef', ascending =False)

In [ ]:
credit_new['education'] = credit_new['education'].str.lower() # Привели значения к нижнему регистру
display(credit_new)

In [ ]:
education_dict = credit_new[['education_id', 'education']]                # Выделили Словари
family_status_dict = credit_new[['family_status_id', 'family_status']]
display(education_dict.head())
display(family_status_dict.head())

### Вывод
Подавбляющие большинство людей, берущих кредиты это средний класс, примерно в 3 раза меньше бедных, в 4 раза меньше богатых и в 10 раз меньше зажиточных.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [ ]:
goal_credit_children_c = credit_new['children'].value_counts()
print(goal_credit_children_c)

In [ ]:
credit_new.pivot_table(index = 'children', columns = 'debt', values = 'education_id', aggfunc = 'count')

In [ ]:
goal_credit_children_debt = credit_new.groupby('children')['debt'].sum()
print(goal_credit_children_debt)

In [ ]:
goal_childre_debt = goal_credit_children_debt / goal_credit_children_c # Посчитаем долю невозвращаемых кредитов каждой категории
print(goal_childre_debt)

In [ ]:
total_children_debt = goal_credit_children_debt.loc[1:].sum() / goal_credit_children_c.loc[1:].sum() # Посчитали сколько % людей с детьми имею просрочек по кредитам ( в независимости от количества детей (1-5))
total_children_debt

In [ ]:
total_children_debt - goal_childre_debt.loc[0] # Посчитали на сколько доля просроченных кредитов у людей с детьми больше тем у бездетных

### Вывод
Люди с детьми в общем имеют больше просроченных кредитов чем те, кто детей не имеет. При этом нет прямой корреляции между количеством детей и начиличем задолженности по кредитам. Следовательно рейтин бездетных людей можно поставить повыше. Возможно это связанно с тем, что у людей с детьми больше непредвиденных трат и обязательных расходов.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [ ]:
goal_credit_family = credit_new['family_status'].value_counts()

In [ ]:
goal_credit_family_debt = credit_new.groupby('family_status')['debt'].sum().sort_values(ascending = False)

In [ ]:
goal_family_debt = goal_credit_family_debt / goal_credit_family
print(goal_family_debt)

In [ ]:
goal_family_debt_2 = 1 - goal_family_debt
goal_family_debt_2

In [ ]:
goal_family_debt_coef = goal_family_debt_2 / 0.934307 # Посчитали коэффициенты значений данного параметра по разным вариантам для модели кредитного скоринга
goal_family_debt_coef

### Вывод
Люди, которые были (или продолжают) связаны с браком имеют меньше просроченных кредитов. Следовательно их рейтинг в кредитном скоринге можно поставить повыше. Возможно люди, знающие что такое брак, лучше научились планировать свои доходы и расходы, а также лучше понимают последствия просрочек по кредитам и в связи с этим чаще своевременно гасят долги. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [ ]:
print(goal_income_debt) # Для наглядности

In [ ]:
print(goal_income)      # Для наглядности

In [ ]:
result_income_debt = goal_income_debt / goal_income
print(result_income_debt)

In [ ]:
qcut_income_pivot['coef'] = qcut_income_pivot[1] / qcut_income_pivot.sum(axis =1) # Втопрой способ категоризации
qcut_income_pivot.sort_values(by = 'coef', ascending =False)

In [ ]:
qcut_income_pivot['1-coef'] = 1 - qcut_income_pivot['coef']            # Посчитали коэффициенты значений данного параметра по разным вариантам для модели кредитного скоринга
qcut_income_pivot['scor_coef'] = qcut_income_pivot['1-coef'] / 0.930136
qcut_income_pivot.sort_values(by = '1-coef', ascending = False)

In [ ]:
qcut_income_pivot['scor_coef'].sort_values(ascending = False)

### Вывод
Люди с большими зарплатами (200000+) реже просрочивают кредиты чем те у кого заработок меньше. Людей с маленьким доходом (меньше 100000) которые имеют просрочки по кредитам примерно на 0,9% больше чем богатых и зажиточных. Чаще всех не успевает вернуть кредит в срок средний класс (доход 100000 - 200000) 8,6 % не возвращают кредиты во время.
Возможно причина в том, что богатые люди имеют меньше просрочек в том, что они лучше умеют обращаться с деньгами, планировать и понимать риски. Возможно бедные имею меньше просроченных задолженностей так как у них каждая копейка на счету, потерять небольшое количество денег (по меркам среднего класса например) для них критично, они бояться и как следствие чаще платят вовремя.
Можно сделать вывод: что рейтинг в кредитном скоринге у богатых и зажиточных групп должен быть выше чем у бедных, а у бедных выше чем у среднего класса за счет их дисциплины гашения.

- Как разные цели кредита влияют на его возврат в срок?

In [ ]:
lemmas_group_count_sum = credit_new.groupby('lemmas_group').agg({'debt': ['count', 'sum']}).sort_values(by = [('debt', 'count')], ascending = False)
print(lemmas_group_count_sum)

In [ ]:
lemmas_group_count_sum['share_debt'] = lemmas_group_count_sum['debt']['sum'] / lemmas_group_count_sum['debt']['count']
lemmas_group_count_sum = lemmas_group_count_sum.sort_values(by = 'share_debt', ascending = False)
print(lemmas_group_count_sum)

In [ ]:
lemmas_group_count_sum['1-share_debt'] = 1 - lemmas_group_count_sum['share_debt']                  # Посчитали коэффициенты значений данного параметра по разным вариантам для модели кредитного скоринга
lemmas_group_count_sum['1-share_debt/0.930957'] = lemmas_group_count_sum['1-share_debt'] / 0.930957
display(lemmas_group_count_sum.sort_values(by = '1-share_debt/0.930957', ascending =False))

In [ ]:
display(lemmas_group_count_sum['1-share_debt/0.930957'].sort_values(ascending =False))

### Вывод
По разным целям кредита разные доли просрочек, просрочки с целью покупки авто и образования имеют максимальную долю (9.2 -9.4%) и примерно равны между собой. Свадьба, недвижимость и жилье имеют соотношение невозврат 7.9%, 7.4% и 6.9% соответственно (шаг в 0.5%). Минимальное колличество просрочек по кредиту на жилье. 
Гипотеза: кридиты на личные покупки (авто и образование) чаще просрачивают чем на совместные и обязательные (свадьба, недвижимость, жилье), следовательно кредитный скоринг во всех аспектах одинаковых кредитов кроме цели должен быть выше у авто и оборазования, далее последовательно идут свадьба, недвижимость, жилье.

### Шаг 4. Общий вывод 

Возврат кредита в срок обратно кореллирует с семейным положением человека (чем ближе он к браку тем меньше просрочек). Богатые и зажиточные люди реже имеют просрочки чем бедные и средний класс (максимальное количество просрочек). Кредиты на глобальные и объединяющие людей цели реже просрачивают. 
Исходя из этого можно скорректировать кредитный скоринг, используя коэффициенты просрочек теми или иными группами людей.

В ходе работы была обработанна, переданная от банка статистика о платёжеспособности клиентов. Были заполненны пропущенные значения, изменен тип данных с float на int, удалены дубликаты, выделены лемы, категоризованны данные, сделаны выводы и сформированы гипотезы по поставленным вопросам (Есть ли зависимость между наличием детей и возвратом кредита в срок? Есть ли зависимость между семейным положением и возвратом кредита в срок? Есть ли зависимость между уровнем дохода и возвратом кредита в срок? Как разные цели кредита влияют на его возврат в срок?).
В изначальных данных было много ошибок: 
- children - были отрицательные и слишком большие значения - (-1 поменяли на 1, 20 поменяли на 2);
- days_emploed - были отрицательные, невозможные и пропущенные значенияж (столбец не нуэен для задания, но были убраны отрицательные значения);
- education - данные написаны в разном регистре - (все данные поменяли на мелкий регистр);
- total_income - были пропущенные значения - (пропущенные значения заполнили медианными значениями по типу занятости);

Есть ли зависимость между наличием детей и возвратом кредита в срок?
Люди с детьми в общем имеют на 1,7% больше просроченных кредитов чем те, кто детей не имеет примерно. При этом нет прямой корреляции между количеством детей и начиличем задолженности по кредитам. Следовательно рейтин бездетных людей можно поставить повыше. Возможно это связанно с тем, что у людей с детьми больше непредвиденных трат и обязательных расходов.

Есть ли зависимость между семейным положением и возвратом кредита в срок?
Люди, которые были (или продолжают) связаны с браком имеют меньше просроченных кредитов. Следовательно их рейтинг в кредитном скоринге можно поставить повыше. Возможно люди, знающие что такое брак, лучше научились планировать свои доходы и расходы, а также лучше понимают последствия просрочек по кредитам и в связи с этим чаще своевременно гасят долги. 
Посчитали коэффициенты значений данного параметра по разным вариантам для модели кредитного скоринга:

женат / замужем          0.989588
гражданский брак         0.970557
Не женат / не замужем    0.965947
в разводе                0.994181
вдовец / вдова           1.000000

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
Люди с большими зарплатами (200000+) реже просрочивают кредиты чем те у кого заработок меньше. Людей с маленьким доходом (меньше 100000) которые имеют просрочки по кредитам примерно на 0,9% больше чем богатых и зажиточных. Чаще всех не успевает вернуть кредит в срок средний класс (доход 100000 - 200000) 8,6 % не возвращают кредиты во время. Возможно причина в том, что богатые люди имеют меньше просрочек в том, что они лучше умеют обращаться с деньгами, планировать и понимать риски. Возможно бедные имею меньше просроченных задолженностей так как у них каждая копейка на счету, потерять небольшое количество денег (по меркам среднего класса например) для них критично, они бояться и как следствие чаще платят вовремя. Можно сделать вывод: что рейтинг в кредитном скоринге у богатых и зажиточных групп должен быть выше чем у бедных, а у бедных выше чем у среднего класса за счет их дисциплины гашения.

Посчитали коэффициенты значений данного параметра по разным вариантам для модели кредитного скоринга:

богатые                  1.000000
бедные                   0.989004
нижний средний класс     0.984728
верхний средний класс    0.984728
средний класс            0.981223

Как разные цели кредита влияют на его возврат в срок?
По разным целям кредита разные доли просрочек, просрочки с целью покупки авто и образования имеют максимальную долю (9.2 -9.4%) и примерно равны между собой. Свадьба, недвижимость и жилье имеют соотношение невозврат 7.9%, 7.4% и 6.9% соответственно (шаг в 0.5%). Минимальное колличество просрочек по кредиту на жилье. Гипотеза: кридиты на личные покупки (авто и образование) чаще просрачивают чем на совместные и обязательные (свадьба, недвижимость, жилье), следовательно кредитный скоринг во всех аспектах одинаковых кредитов кроме цели должен быть выше у авто и оборазования, далее последовательно идут свадьба, недвижимость, жилье.

Посчитали коэффициенты значений данного параметра по разным вариантам для модели кредитного скоринга:

жилье           1.000000
недвижимость    0.994020
свадьба         0.988598
образование     0.975150
автомобиль      0.973679